In [1]:
from predict import change_spacing
import nibabel as nib
import numpy as np
import multiprocessing
import datetime
import sys
import os

In [2]:
proc_num = 6

In [3]:
dataset_dir = "/media/eolika/8224BB4D24BB42C9/BLUEMIND/nnUNet/trains/nnUNet_raw/Dataset101_mosaic_bones_and_muscles"
dataset_imgs_dir = os.path.join(dataset_dir, 'imagesTr')
dataset_lbls_dir = os.path.join(dataset_dir, 'labelsTr')

len(os.listdir(dataset_imgs_dir)), len(os.listdir(dataset_lbls_dir))

(2224, 2224)

In [4]:
def update_progress():
    with processed_count.get_lock():
        processed_count.value += 1
    progress = processed_count.value / case_count * 100
    elapsed_time = (datetime.datetime.now() - start_time).total_seconds()
    remaining_time = (elapsed_time / processed_count.value) * (case_count - processed_count.value)
    progress_str = f"Progress: {progress:.2f}% ({processed_count.value}/{case_count})"
    time_str = f"Elapsed Time: {str(datetime.timedelta(seconds=int(elapsed_time)))} | Remaining Time: {str(datetime.timedelta(seconds=int(remaining_time)))}"
    output_str = f"{progress_str.ljust(30)} | {time_str}"
    sys.stdout.write('\r' + output_str)
    sys.stdout.flush()

def process_general_case(parent_dir, case_folder, order, dtype):
    try:
        input_file = os.path.join(parent_dir, case_folder)

        img_in = nib.load(input_file)

        img_in_rsp = change_spacing(img_in, 
                                    [1., 1., 1.],
                                    order=order, 
                                    dtype=dtype, 
                                    nr_cpus=1)

        nib.save(img_in_rsp, input_file)
    except Exception as e:
        print(e)
        exception_cases.append(case_folder)


### images

In [ ]:
case_folders = os.listdir(dataset_imgs_dir)

case_count = len(case_folders)
processed_count = multiprocessing.Value('i', 0)  # Shared counte

start_time = datetime.datetime.now()
prev_elapsed_time = 0

exception_cases = []

# multi cycle magic
pool = multiprocessing.Pool(processes=proc_num)
i = 0
for case_folder in os.listdir(dataset_imgs_dir):
    pool.apply_async(process_general_case, args=(dataset_imgs_dir, case_folder, 3, np.int32), callback=lambda _: update_progress())

pool.close()
pool.join()

print(f'\nDone! Exception cases: {exception_cases}')
print(f'Files in imagesTr: {len(os.listdir(dataset_imgs_dir))}')
print(f'Files in labelsTr: {len(os.listdir(dataset_lbls_dir))}')

### labels

In [5]:
case_folders = os.listdir(dataset_lbls_dir)

case_count = len(case_folders)
processed_count = multiprocessing.Value('i', 0)  # Shared counte

start_time = datetime.datetime.now()
prev_elapsed_time = 0

exception_cases = []

# multi cycle magic
pool = multiprocessing.Pool(processes=proc_num)
i = 0
for case_folder in os.listdir(dataset_lbls_dir):
    pool.apply_async(process_general_case, args=(dataset_lbls_dir, case_folder, 0, np.uint8), callback=lambda _: update_progress())

pool.close()
pool.join()

print(f'\nDone! Exception cases: {exception_cases}')
print(f'Files in imagesTr: {len(os.listdir(dataset_imgs_dir))}')
print(f'Files in labelsTr: {len(os.listdir(dataset_lbls_dir))}')

Progress: 100.00% (2224/2224)  | Elapsed Time: 0:11:26 | Remaining Time: 0:00:00
Done! Exception cases: []
Files in imagesTr: 2224
Files in labelsTr: 2224


In [6]:
import nibabel as nib

img_ct = nib.load("/media/eolika/8224BB4D24BB42C9/BLUEMIND/nnUNet/trains/nnUNet_raw/Dataset101_mosaic_bones_and_muscles/imagesTr/botkin_0003_0000.nii.gz")
img_seg = nib.load("/media/eolika/8224BB4D24BB42C9/BLUEMIND/nnUNet/trains/nnUNet_raw/Dataset101_mosaic_bones_and_muscles/labelsTr/botkin_0003.nii.gz")

In [7]:
img_ct.header.get_zooms()

(1.0, 1.0, 1.0)

In [8]:
img_seg.header.get_zooms()

(1.0, 1.0, 1.0)